In [ ]:
##############

CLASS_NUM = 115 # size of class for classifiy
num_feature = 128 # embedding vector size
PCA_CUM_RATIO = 0.85 # PCA cumulative proportion
PCA = False 
OUTLIER = False
UNDERSAMPLING = False

##############

In [ ]:
# crawling한 excel file 불러오기 
from google.colab import drive
drive.mount("gdrive")

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


In [ ]:
from glob import glob
sheet_list = glob('/content/gdrive/My Drive/Colab Notebooks/DeepAgency/emb/*.npy')

In [ ]:
import numpy as np
from sklearn.ensemble import IsolationForest

In [ ]:
min_len = 99999

if (UNDERSAMPLING == True):
  for i in range(0, CLASS_NUM):
    tmp = np.load(sheet_list[i])
    len = tmp.shape[0]

    if(min_len > len):
      min_len = len

In [ ]:
train_data = []
label = []

for i in range(0, CLASS_NUM):
  tmp = np.load(sheet_list[i])
  #globals()['Sheet' + str(i+1)] = tmp

  if (OUTLIER == True):
    #outlier 제거 
    clf=IsolationForest()
    clf.fit(tmp[:,1:])
    pred = clf.predict(tmp[:,1:])
    tmp = tmp[np.where(pred == 1)]

  if (UNDERSAMPLING == True):
    tmp = tmp[np.random.choice(tmp.shape[0], min_len, replace = False)]
    
  label.extend(tmp[:,0])
  train_data.extend(tmp[:,1:])

In [ ]:
np.array(train_data).shape 

(80591, 128)

In [ ]:
np.array(label).shape # (80591, )

(80591,)

In [ ]:
## PCA
from sklearn.decomposition import PCA

In [ ]:
if PCA == True:
  pca = PCA(n_components=num_feature)
  PC_score = pca.fit_transform(train_data)

  #pca.explained_variance_

  cumsum = np.cumsum(pca.explained_variance_ratio_)
  d = np.argmax(cumsum >= PCA_CUM_RATIO) + 1

  print('선택할 차원 수 :', d)

  PC_score[:,:d]

In [ ]:
from sklearn.model_selection import train_test_split

if PCA == True:
    df_train = PC_score[:,:d]
    print("PCA done")
X_tr, X_vld, y_tr, y_vld = train_test_split(np.array(train_data), np.array(label), stratify = label, test_size=0.1, random_state=2020)

In [ ]:
from keras.models import Sequential 
from keras.layers import Dense , Dropout, Flatten
from keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from keras.optimizers import RMSprop, Adam, Adadelta
import random 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import metrics 

In [ ]:
def create_model( lr = 0.01, opt = 'Adam', init = 'glorot_uniform', dr = 0.4): # create model 

  # fix random seed for reproducibility
  seed = 42 
  random.seed(seed) 

  model = Sequential()
  
  model.add(Dense(1024, input_dim=X_tr.shape[1], activation='relu', kernel_initializer =init)) 
  model.add(Dropout(dr))
  model.add(Dense(512, activation='relu'))


  model.add(Dense(CLASS_NUM, activation='softmax')) 

  #opt= rmsprop(lr=lr)
  # Compile model
  model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy']) 
  #loss for categorical_crossentropy -> one hot code 

  return model

In [ ]:
epoch_size = 50
batch_size = 16

In [ ]:
nnmodel = KerasClassifier(build_fn=create_model, epochs=epoch_size, batch_size=batch_size, verbose=1 ,validation_split=0.1)

In [ ]:
# from sklearn.model_selection import cross_val_score

# accuracies =  cross_val_score(estimator=nnmodel, X= X_tr, y=y_tr, cv=10, n_jobs=-1)
# accuracies
# accuracies.mean()

| hidden_layer1 | hidden_layer_2 | result |   
|---|:---:|---:|
| **512** | **64** | **0.7297569692134858**|
| **256** | **64** | **0.7298671960830688**|
| **128** | **64** | **0.7028168261051178**|
| **512** | **128** | **0.7346376538276672**|
| **256** | **128** | **0.7263929426670075**|
| **512** | **256** | **0.7368022680282593**|
| **1024** | **512** | **0.732638555765152**|
| **1024** | **256** | **0.7347892940044403**|

In [ ]:
history = nnmodel.fit(X_tr, y_tr, callbacks = [])
nnprediction = np.argmax(nnmodel.predict(X_vld), axis=-1) 
#nnprediction = nnprediction.flatten()

Epoch 1/50
4080/4080 [==============================] - 11s 3ms/step - loss: 1.6603 - accuracy: 0.6173 - val_loss: 1.3263 - val_accuracy: 0.6864
Epoch 2/50
4080/4080 [==============================] - 11s 3ms/step - loss: 1.2599 - accuracy: 0.6987 - val_loss: 1.2127 - val_accuracy: 0.7098
Epoch 3/50
4080/4080 [==============================] - 11s 3ms/step - loss: 1.1348 - accuracy: 0.7206 - val_loss: 1.1581 - val_accuracy: 0.7244
Epoch 4/50
4080/4080 [==============================] - 11s 3ms/step - loss: 1.0475 - accuracy: 0.7373 - val_loss: 1.1440 - val_accuracy: 0.7272
Epoch 5/50
2947/4080 [====================>.........] - ETA: 2s - loss: 0.9649 - accuracy: 0.7518

In [ ]:
loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(1)
for l in acc_list:
    plt.plot(range(epoch_size), history.history[l], 'b')
for l in val_acc_list:
    plt.plot(range(epoch_size), history.history[l], 'g')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')


plt.figure(2)
for l in loss_list:
    plt.plot(range(epoch_size), history.history[l], 'b')
for l in val_loss_list:
    plt.plot(range(epoch_size), history.history[l], 'g')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
nnmodel.model.save('predict.h5')

In [ ]:
import tensorflow as tf

saved_model_dir = '/content/predict.h5'

keras_model = tf.keras.models.load_model(saved_model_dir)
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
buffer = converter.convert()
open('predict.tflite', 'wb').write(buffer)

INFO:tensorflow:Assets written to: /tmp/tmp2jzfayti/assets


INFO:tensorflow:Assets written to: /tmp/tmp2jzfayti/assets


587916

#### optimize

In [ ]:
# using Grid Search to optimize hyper-parms
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=create_model, verbose=0)

epochs = [10] ##// increase epochs        
batches = [1,4,8]

param_grid = dict(epochs=epochs, batch_size=batches)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_tr, y_tr)                                                                                                                                    

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score'] 
stds = grid_result.cv_results_['std_test_score']

params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

KeyboardInterrupt: ignored

In [ ]:
# # using Grid Search to optimize hyper-parms
# model = KerasClassifier(build_fn=create_model,epochs=epoch_size, batch_size=batch_size ,verbose=0)

# optimizers = ['RMSprop', 'Adam' ,'Adadelta']
# init = ['glorot_uniform', 'he_normal']

# param_grid = dict(opt=optimizers, init=init)                                                                                                                                 
# grid = GridSearchCV(estimator=model, param_grid=param_grid)
# grid_result = grid.fit(X_tr, y_tr)   

# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score'] 
# stds = grid_result.cv_results_['std_test_score']

# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
# 	print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# # using Grid Search to optimize hyper-parms
# model = KerasClassifier(build_fn=create_model,epochs=epoch_size, batch_size=batch_size ,verbose=0)

# drops = [0.0, 0.1, 0.2, 0.3, 0.4]

# param_grid = dict(dr = drops)                                                                                                                                 
# grid = GridSearchCV(estimator=model, param_grid=param_grid)
# grid_result = grid.fit(X_tr, y_tr)   

# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score'] 
# stds = grid_result.cv_results_['std_test_score']

# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
# 	print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
# from sklearn.model_selection import cross_val_score

# accuracies =  cross_val_score(estimator=nnmodel, X= X_tr, y=y_tr, cv=10, n_jobs=-1)
# accuracies
# accuracies.mean()

| hidden_layer1 | hidden_layer_2 | result |   
|---|:---:|---:|
| **512** | **32** | **0.040794732980430126**|
| **256** | **32** | **0.040920873358845714**|
| **128** | **32** | **0.035837072692811486**|
| **64** | **32** | **0.043054659850895405**|
| **512** | **64** | **0.04437438491731882**|
| **256** | **64** | **0.040669419802725315**|
| **128** | **64** | **0.04343150444328785**|
| **512** | **128** | **0.0374689768999815**|
| **256** | **128** | **0.037028411589562896**|
| **512** | **256** | **0.036026145052164796**|